# Residual model generation pipeline

In [ ]:
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt

from LMCE.model import DTE
import LMCE.cfusdlog as cfusdlog
from LMCE.data_prep import prepare_data
import LMCE.uav_trajectory as uav_trajectory
from LMCE.residual_calculation import residual
from LMCE.model_to_c_conversion import exportTree, tree_c_model_test
from LMCE.error_estimation import error_calculator, find_best_cutoff

plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times']

## Preparing the data
First define where the data is stored, then it can be loaded, the `prepare_data` function calculates the labels (residual forces).

In [ ]:
# Get train data paths
indices = [i for i in range(0, 86)]
test_indices = [2]
for i in test_indices:
    del indices[i]
training_data_paths = [f"./crazyflie-data-collection/jana_flight_data/jana{i:02}" for i in indices]
testing_data_paths = [f"./crazyflie-data-collection/jana_flight_data/jana{i:02}" for i in test_indices]

# training_data_paths = [f"./crazyflie-data-collection/brushless_flights/data/eckart{i:02}" for i in indices]
# testing_data_paths = [f"./crazyflie-data-collection/brushless_flights/data/eckart{i:02}" for i in test_indices]

# Prepare data
residual_func = lambda data: residual(data)
# residual_func = lambda data: residual(data, is_brushless=True, has_payload=False, total_mass=.0444, use_rpm=True)
X_train, y_train = prepare_data(training_data_paths, save_as="train_data", residual_func=residual_func, overwrite=True)
X_test, y_test = prepare_data(testing_data_paths, save_as="test_data", shuffle_data=False, residual_func=residual_func, overwrite=True)

We can plot the flight path and residual forces like this

In [ ]:
data_path = testing_data_paths[0]
visual_scale = 1.

fig = plt.figure()
ax = plt.axes(projection='3d')

data = cfusdlog.decode(data_path)['fixedFrequency']
label = os.path.basename(data_path)

x = [i for i in data["stateEstimate.x"]]
y = [i for i in data["stateEstimate.y"]]
z = [i for i in data["stateEstimate.z"]]
origin = np.array([x, y, z]).T

f, _ = residual_func(data)
vector = np.array([f[:, 0], f[:, 1], f[:, 2]]).T * visual_scale
ax.quiver(origin[:, 0], origin[:, 1], origin[:, 2], vector[:, 0], vector[:, 1],
          vector[:, 2], color='r', alpha=.1, label=f"Residual forces (scaled by {visual_scale})")

ax.plot3D(x, y, z, label=label)

# Calculate the avg. residual force for each axis
for j, v in enumerate(["x", "y", "z"]):
    f_j = [f_[j] for f_ in f]
    print(f"{label} mean residual f_{v}: {sum(f_j)/len(f_j)}")

plt.title("Residual forces on quadrotor trajectory")
plt.legend()
plt.axis('equal')
plt.show()

## Training the model

In [ ]:
model = DTE()
model.train(X_train, y_train)
model.save("dte")

## Testing the model

In [ ]:
model.load("dte")

pred = model.forward(X_test)
error = np.abs(y_test-pred)
print(f"Average error: (x) -> {np.mean(error[:, 0])} (y) -> {np.mean(error[:, 1])}")

fig, ax = plt.subplots(2, figsize=(10, 8))

ax[0].plot(y_test[:, 0], label="Real")
ax[0].plot(pred[:, 0], label="Predicted")
ax[0].set_title('Force X')
ax[0].set_xlabel('Timestamp')
ax[0].set_ylabel('Force prediction')
ax[0].legend()

ax[1].plot(y_test[:, 1], label="Real")
ax[1].plot(pred[:, 1], label="Predicted")
ax[1].set_title('Force Y')
ax[1].set_xlabel('Timestamp')
ax[1].set_ylabel('Force prediction')

plt.tight_layout(rect=[0, 0, 1, 0.95])

plt.savefig("./old_scripts/imgs/tree_inf.png", dpi=600)
plt.show()

## Translating the model to c code

In [ ]:
# Generate the model c code
model_path = "./models/dte.pkl"
exportTree(model_path, info="Neural network for the standard Crazyflie")

# Test if the generated model gives the same outputs as the original
tree_c_model_test(model, "./data/test_data_payload.npz")

## Adding the model to the cf-firmware

In [ ]:
shutil.copyfile("./LMCE/c_utils/tree.c", "./crazyflie-firmware/src/modules/src/controller/tree.c")
shutil.copyfile("./LMCE/c_utils/tree.h", "./crazyflie-firmware/src/modules/interface/controller/tree.h")

## Error calculation
For calculating the error with the addition of the learning models the flights need to be re-recorded with the updated firmware.

In [ ]:
# Get desired path
traj = uav_trajectory.Trajectory()
traj.loadcsv("./LMCE/flight_paths/figure8.csv")
traj.stretchtime(2)

ts = np.arange(0, traj.duration, 0.01)
evals = np.empty((len(ts), 15))
for t, i in zip(ts, range(0, len(ts))):
    e = traj.eval(t)
    e.pos += np.array([0, 0, 1])
    evals[i, 0:3] = e.pos

target_pos = evals.transpose()

# Get real path
data_path = testing_data_paths[0]
data = cfusdlog.decode(data_path)['fixedFrequency']
real_pos = [data["stateEstimate.x"], data["stateEstimate.y"]]

# Calculate error
cutoff = find_best_cutoff(real_pos, target_pos)
avg_error = error_calculator(cutoff, real_pos, target_pos, vis=True)
print(f"Avg. error: {avg_error:.4f} (m)")